In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from IPython.display import display
sb.set() # set the default Seaborn style for graphics

# PROBLEM STATEMENT:

# Most important variable in retaining high Customer Life Value (CLV) customers

## Importing Datasets

In [2]:
customerdata = pd.read_csv("olist_customers_dataset.csv")
orderitems = pd.read_csv("olist_order_items_dataset.csv")
orderpayments = pd.read_csv("olist_order_payments_dataset.csv")
orderreviews = pd.read_csv("olist_order_reviews_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
translation = pd.read_csv("product_category_name_translation.csv")
datasetlist = [orderitems,orderpayments,orderreviews,orders,products,translation]

## Basic Data Exploration

In [3]:
for x in datasetlist:
    x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 

## Checking for Discrepancies

In [4]:
print(len(orderitems.order_id.unique()))
print(len(orders.order_id.unique()))
print(len(orderpayments.order_id.unique()))
print(len(orderreviews.order_id.unique()))

98666
99441
99440
99441


### Missing values of order_id found between the datasets. Filter out the missing values and find out the reason why they are dropped between datasets. 

In [5]:
#Finding list of missing order_id
missinglist = []
testset = set(orderitems.order_id)
for x in orders.order_id:
    if x in testset:
        continue
    else:
        missinglist.append(x)
        

#Extracting missing data from orders dataset
missingdf = pd.DataFrame
missingdf = orders[orders['order_id'].isin(missinglist)]
missingdf.info()
missingdf

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 266 to 99415
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       775 non-null    object
 1   customer_id                    775 non-null    object
 2   order_status                   775 non-null    object
 3   order_purchase_timestamp       775 non-null    object
 4   order_approved_at              629 non-null    object
 5   order_delivered_carrier_date   1 non-null      object
 6   order_delivered_customer_date  0 non-null      object
 7   order_estimated_delivery_date  775 non-null    object
dtypes: object(8)
memory usage: 54.5+ KB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaN,NaN,2017-12-05 00:00:00
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaN,NaN,2018-02-16 00:00:00
687,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaN,NaN,2017-09-05 00:00:00
737,d57e15fb07fd180f06ab3926b39edcd2,470b93b3f1cde85550fc74cd3a476c78,unavailable,2018-01-08 19:39:03,2018-01-09 07:26:08,NaN,NaN,2018-02-06 00:00:00
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,NaN,NaN,NaN,2018-09-12 00:00:00
...,...,...,...,...,...,...,...,...
99252,aaab15da689073f8f9aa978a390a69d1,df20748206e4b865b2f14a5eabbfcf34,unavailable,2018-01-16 14:27:59,2018-01-17 03:37:34,NaN,NaN,2018-02-06 00:00:00
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaN,NaN,NaN,2018-09-27 00:00:00
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaN,NaN,2017-09-15 00:00:00


### Missing order_ids found to be cancelled / unsuccessful orders. As such, we will drop these orders from all datasets.

In [6]:
neworders = orders[~orders.order_id.isin(missinglist)]
neworderpayments = orderpayments[~orderpayments.order_id.isin(missinglist)]
neworderreviews = orderreviews[~orderreviews.order_id.isin(missinglist)]

print(len(orderitems.order_id.unique()))
print(len(neworders.order_id.unique()))
print(len(neworderpayments.order_id.unique()))
print(len(neworderreviews.order_id.unique()))

98666
98666
98665
98666


### There is still one missing value in order payments. Let's find out

In [7]:
missinglist = []
testset = set(neworderpayments.order_id)
for x in orderitems.order_id:
    if x in testset:
        continue
    else:
        missinglist.append(x)

#Extracting missing data from orders dataset
missingdf = pd.DataFrame
missingdf = neworderreviews[neworderreviews['order_id'].isin(missinglist)]
missingdf

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
37828,6916ca4502d6d3bfd39818759d55d536,bfbd0f9bdef84302105ad712db648a6c,1,NaN,nao recebi o produto e nem resposta da empresa,2016-10-06 00:00:00,2016-10-07 18:32:28


##### "nao recebi o produto e nem resposta da empresa" =  I did not receive the product and no response from the company
##### So we will drop this order_id as well

##### Seems to have some missing values in delivery date under orders dataset as well

In [8]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


#### Presence of NA values in column 5 and 6. As such, we will drop those too.

In [9]:
miss = orders[orders['order_delivered_customer_date'].isna() | orders['order_delivered_carrier_date'].isna() | orders['order_approved_at'].isna()]
missing = miss['order_id'].tolist() + missinglist 

In [10]:
newerorders = neworders[~neworders.order_id.isin(missing)]
newerorderpayments = neworderpayments[~neworderpayments.order_id.isin(missing)]
newerorderreviews = neworderreviews[~neworderreviews.order_id.isin(missing)]
newerorderitems = orderitems[~orderitems.order_id.isin(missing)]

print(len(newerorderitems.order_id.unique()))
print(len(newerorders.order_id.unique()))
print(len(newerorderpayments.order_id.unique()))
print(len(newerorderreviews.order_id.unique()))

96460
96460
96460
96460


### Now we will concat these dataframes together

In [11]:
newerorderitems.info()
newerorders.info()
newerorderpayments.info()
newerorderreviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110177 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             110177 non-null  object 
 1   order_item_id        110177 non-null  int64  
 2   product_id           110177 non-null  object 
 3   seller_id            110177 non-null  object 
 4   shipping_limit_date  110177 non-null  object 
 5   price                110177 non-null  float64
 6   freight_value        110177 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.7+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96460 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       96460 non-null  object
 1   customer_id                    96460 non-null  object
 2   order_status        

In [12]:
new = pd.merge(newerorderitems, newerorderpayments, on='order_id')
new1 = pd.merge(new, newerorderreviews, on='order_id')
new2 = pd.merge(new1, newerorders, on='order_id')
completedata = pd.merge(new2, products, on='product_id')

# translate category names to english
translate = translation.set_index('product_category_name').to_dict()['product_category_name_english']

completedata['product_category_name'].replace(translate, inplace=True)

print(completedata["product_category_name"])

0              cool_stuff
1              cool_stuff
2              cool_stuff
3              cool_stuff
4              cool_stuff
               ...       
115706       garden_tools
115707    furniture_decor
115708      watches_gifts
115709     sports_leisure
115710     bed_bath_table
Name: product_category_name, Length: 115711, dtype: object


In [13]:
completedata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115711 entries, 0 to 115710
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115711 non-null  object 
 1   order_item_id                  115711 non-null  int64  
 2   product_id                     115711 non-null  object 
 3   seller_id                      115711 non-null  object 
 4   shipping_limit_date            115711 non-null  object 
 5   price                          115711 non-null  float64
 6   freight_value                  115711 non-null  float64
 7   payment_sequential             115711 non-null  int64  
 8   payment_type                   115711 non-null  object 
 9   payment_installments           115711 non-null  int64  
 10  payment_value                  115711 non-null  float64
 11  review_id                      115711 non-null  object 
 12  review_score                  

## Still have some null values in product category, description, etc. We will drop all these rows.

In [14]:
finaldata = completedata.dropna(subset = ['product_category_name','product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm',])
finaldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114073 entries, 0 to 115710
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       114073 non-null  object 
 1   order_item_id                  114073 non-null  int64  
 2   product_id                     114073 non-null  object 
 3   seller_id                      114073 non-null  object 
 4   shipping_limit_date            114073 non-null  object 
 5   price                          114073 non-null  float64
 6   freight_value                  114073 non-null  float64
 7   payment_sequential             114073 non-null  int64  
 8   payment_type                   114073 non-null  object 
 9   payment_installments           114073 non-null  int64  
 10  payment_value                  114073 non-null  float64
 11  review_id                      114073 non-null  object 
 12  review_score                  

### Now we are left with null values only present in the review title and message. We will not be dropping these values

# Calculation of Retention Rates:
### If a customer orders more than once, we will assume that the customer is a "retained customer"
### We will then use a few predictors to determine which variable is most important that impacts the categorical Yes/No of Churn.

In [15]:
len(finaldata.customer_id.unique())

95128

"At our system each order is assigned to a unique customerid. This means that the same customer will get different ids for different orders. The purpose of having a customerunique_id on the dataset is to allow you to identify customers that made repurchases at the store." 
### We would need to add a new column, customerunique_id to the orders dataset to identify repeat customers

In [23]:
customer_translate = customerdata.set_index('customer_id').to_dict()['customer_unique_id']
#finaldata['customer_id'].map(customer_translate)
finaldata['customer_id'].replace(customer_translate, inplace=True)
print(len(finaldata.customer_id.unique()))

1


C:\Users\bawj\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
